# Extracting the data out of the zip

In [83]:
import zipfile
import json
import sqlite3
import pandas as pd
from pathlib import Path

conn = sqlite3.connect("scopus.db")
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS papers_raw (
    file_id INTEGER,
    year INTEGER,
    raw_json TEXT
)
""")

conn.commit()
conn.close()

keep = "ScopusData2018-2023/2018/201800000"
zip_path = "ScopusData2018-2023.zip"
years = ["2018", "2019", "2020", "2021", "2022", "2023"]

def loader(year: int, start_id: int, end_id: int):
    conn = sqlite3.connect("scopus.db")
    cur = conn.cursor()

    with zipfile.ZipFile(zip_path, "r") as z:
        for file_id in range(start_id, end_id + 1):
            inner_path = f"ScopusData2018-2023/{year}/{file_id}"
            try:
                with z.open(inner_path) as f:
                    try:
                        obj = json.load(f)
                    except Exception:
                        continue
            except KeyError:
                continue
            raw_text = json.dumps(obj, ensure_ascii=False)
            cur.execute(
                """
                INSERT INTO papers_raw (file_id, year, raw_json)
                VALUES (?, ?, ?)
                """,
                (file_id, year, raw_text)
            )
    conn.commit()
    conn.close()




In [84]:
sql_data_path = Path("scopus.db")
if not sql_data_path.exists():

    # 2018
    loader(2018, 201800000, 201802761)

    # 2019
    loader(2019, 201900000, 201903081)

    # 2020
    loader(2020, 202000000, 202003392)

    # 2021
    loader(2021, 202100000, 202103814)

    # 2022
    loader(2022, 202200000, 202204243)

    # 2023
    loader(2023, 202300000, 202302889)
else:
    print("scopus.db already exists")

scopus.db already exists


In [85]:
conn = sqlite3.connect("scopus.db")

df_2018 = pd.read_sql_query(
    """
    SELECT file_id, year, raw_json
    FROM papers_raw
    WHERE year = 2018
    """,
    conn
)

conn.close()

print(df_2018.shape)
print(df_2018.head())

(2762, 3)
     file_id  year                                           raw_json
0  201800000  2018  {"abstracts-retrieval-response": {"item": {"ai...
1  201800001  2018  {"abstracts-retrieval-response": {"item": {"ai...
2  201800002  2018  {"abstracts-retrieval-response": {"item": {"ai...
3  201800003  2018  {"abstracts-retrieval-response": {"item": {"ai...
4  201800004  2018  {"abstracts-retrieval-response": {"item": {"ai...


In [86]:
import sqlite3, pandas as pd
if("df" not in globals()):
    conn = sqlite3.connect("scopus.db")

    SQL = """
    SELECT
      file_id,
      year,

      -- basics
      json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head."citation-title"') AS citation_title,
      json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head.abstracts')        AS abstracts,
      json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head.source.publisher.publishername') AS publishername,
      json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head.source.sourcetitle')             AS sourcetitle,

      /* publication_date: DD/MM/YYYY from day, month, year (NULL if any part missing) */
      CASE
        WHEN json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head.source.publicationdate.day')   IS NOT NULL
         AND json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head.source.publicationdate.month') IS NOT NULL
         AND json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head.source.publicationdate.year')  IS NOT NULL
        THEN printf('%02d/%02d/%04d',
                    CAST(json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head.source.publicationdate.day')   AS INTEGER),
                    CAST(json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head.source.publicationdate.month') AS INTEGER),
                    CAST(json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head.source.publicationdate.year')  AS INTEGER))
        ELSE NULL
      END AS publication_date,

      /* ce:doi pulled then aliased as document_classification_codes */
      COALESCE(
        json_extract(raw_json,'$."abstracts-retrieval-response".item."item-info"."itemidlist"."ce:doi"'),
        (SELECT t.value
         FROM json_tree(raw_json, '$."abstracts-retrieval-response"') AS t
         WHERE t.key = 'ce:doi'
         LIMIT 1)
      ) AS document_classification_codes,

      -- counts
      json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.tail.bibliography."@refcount"') AS refcount,
      CAST(
        COALESCE(
          json_extract(raw_json,'$."abstracts-retrieval-response".coredata."citedby-count"'),
          json_extract(raw_json,'$."abstracts-retrieval-response".item.coredata."citedby-count"'),
          (SELECT t.value
           FROM json_tree(raw_json, '$."abstracts-retrieval-response"') AS t
           WHERE t.key = 'citedby-count'
           LIMIT 1)
        ) AS INTEGER
      ) AS citedbycount,

      /* authors (degree + given + surname) as JSON array of strings */
      (
        SELECT json_group_array(name_str)
        FROM (
          SELECT
            TRIM(
              COALESCE(json_extract(a.value,'$."ce:degrees"') || ' ', '') ||
              TRIM(
                COALESCE(json_extract(a.value,'$."preferred-name"."ce:given-name"'),
                         json_extract(a.value,'$."ce:given-name"'), '') || ' ' ||
                COALESCE(json_extract(a.value,'$."preferred-name"."ce:surname"'),
                         json_extract(a.value,'$."ce:surname"'), '')
              )
            ) AS name_str
          FROM json_each(
                 COALESCE(
                   json_extract(raw_json,'$."abstracts-retrieval-response".authors.author'),
                   json_array()
                 )
               ) AS a
          WHERE TRIM(
                  COALESCE(json_extract(a.value,'$."preferred-name"."ce:given-name"'),
                           json_extract(a.value,'$."ce:given-name"'), '') || ' ' ||
                  COALESCE(json_extract(a.value,'$."preferred-name"."ce:surname"'),
                           json_extract(a.value,'$."ce:surname"'), '')
                ) <> ''
        )
      ) AS authors_deg_name_json,

      /* categories (subject → abbrev) as JSON array of {"<name>":"<abbrev>"} */
      (
        SELECT json_group_array(json_object(subject, abbrev))
        FROM (
          SELECT DISTINCT
            json_extract(sa.value,'$."$"')        AS subject,
            json_extract(sa.value,'$."@abbrev"')  AS abbrev
          FROM (
            -- primary
            SELECT * FROM json_each(
              CASE json_type(json_extract(raw_json,'$."abstracts-retrieval-response"."subject-areas"."subject-area"'))
                WHEN 'array'  THEN json_extract(raw_json,'$."abstracts-retrieval-response"."subject-areas"."subject-area"')
                WHEN 'object' THEN json_array(json_extract(raw_json,'$."abstracts-retrieval-response"."subject-areas"."subject-area"'))
                ELSE json_array()
              END
            )
            UNION ALL
            -- fallback A
            SELECT * FROM json_each(
              CASE json_type(json_extract(raw_json,'$."abstracts-retrieval-response".item.coredata."subject-areas"."subject-area"'))
                WHEN 'array'  THEN json_extract(raw_json,'$."abstracts-retrieval-response".item.coredata."subject-areas"."subject-area"')
                WHEN 'object' THEN json_array(json_extract(raw_json,'$."abstracts-retrieval-response".item.coredata."subject-areas"."subject-area"'))
                ELSE json_array()
              END
            )
            UNION ALL
            -- fallback B
            SELECT * FROM json_each(
              CASE json_type(json_extract(raw_json,'$."abstracts-retrieval-response".coredata."subject-areas"."subject-area"'))
                WHEN 'array'  THEN json_extract(raw_json,'$."abstracts-retrieval-response".coredata."subject-areas"."subject-area"')
                WHEN 'object' THEN json_array(json_extract(raw_json,'$."abstracts-retrieval-response".coredata."subject-areas"."subject-area"'))
                ELSE json_array()
              END
            )
          ) AS sa
          WHERE subject IS NOT NULL AND abbrev IS NOT NULL
        )
      ) AS categories,

      /* creator = "<given> <surname>" (first creator found) */
      (
        SELECT name_full
        FROM (
          SELECT TRIM(
                   COALESCE(json_extract(a.value,'$."preferred-name"."ce:given-name"'),
                            json_extract(a.value,'$."ce:given-name"'), '') || ' ' ||
                   COALESCE(json_extract(a.value,'$."preferred-name"."ce:surname"'),
                            json_extract(a.value,'$."ce:surname"'), '')
                 ) AS name_full
          FROM (
            SELECT * FROM json_each(
              CASE json_type(json_extract(raw_json,'$."abstracts-retrieval-response".item.coredata."dc:creator".author'))
                WHEN 'array'  THEN json_extract(raw_json,'$."abstracts-retrieval-response".item.coredata."dc:creator".author')
                WHEN 'object' THEN json_array(json_extract(raw_json,'$."abstracts-retrieval-response".item.coredata."dc:creator".author'))
                ELSE json_array()
              END
            )
            UNION ALL
            SELECT * FROM json_each(
              CASE json_type(json_extract(raw_json,'$."abstracts-retrieval-response".coredata."dc:creator".author'))
                WHEN 'array'  THEN json_extract(raw_json,'$."abstracts-retrieval-response".coredata."dc:creator".author')
                WHEN 'object' THEN json_array(json_extract(raw_json,'$."abstracts-retrieval-response".coredata."dc:creator".author'))
                ELSE json_array()
              END
            )
          ) AS a
          WHERE name_full <> ''
          LIMIT 1
        )
      ) AS creator,

      /* ---------- NEW: keywords (JSON array of "$" strings) ---------- */
      (
        SELECT json_group_array(kw_src.kw)
        FROM (
          -- Path 1: head → citation-info → author-keywords
          SELECT json_extract(k.value,'$."$"') AS kw
          FROM json_each(
            CASE json_type(json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head."citation-info"."author-keywords"."author-keyword"'))
              WHEN 'array'  THEN json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head."citation-info"."author-keywords"."author-keyword"')
              WHEN 'object' THEN json_array(json_extract(raw_json,'$."abstracts-retrieval-response".item.bibrecord.head."citation-info"."author-keywords"."author-keyword"'))
              ELSE json_array()
            END
          ) AS k
          UNION ALL
          -- Path 2: top-level authkeywords
          SELECT json_extract(k2.value,'$."$"') AS kw
          FROM json_each(
            CASE json_type(json_extract(raw_json,'$."abstracts-retrieval-response"."authkeywords"."author-keyword"'))
              WHEN 'array'  THEN json_extract(raw_json,'$."abstracts-retrieval-response"."authkeywords"."author-keyword"')
              WHEN 'object' THEN json_array(json_extract(raw_json,'$."abstracts-retrieval-response"."authkeywords"."author-keyword"'))
              ELSE json_array()
            END
          ) AS k2
        ) AS kw_src
        WHERE kw_src.kw IS NOT NULL
      ) AS keywords

    FROM papers_raw
    ORDER BY year, file_id;
    """

    df = pd.read_sql_query(SQL, conn)
else:
    print("df already exists")



df already exists


# EDA

In [87]:
for c in df:
    print(df[c].describe())

count    2.018600e+04
mean     2.020630e+08
std      1.627104e+05
min      2.018000e+08
25%      2.019023e+08
50%      2.021009e+08
75%      2.022021e+08
max      2.023029e+08
Name: file_id, dtype: float64
count    20186.000000
mean      2020.612652
std          1.625812
min       2018.000000
25%       2019.000000
50%       2021.000000
75%       2022.000000
max       2023.000000
Name: year, dtype: float64
count       20185
unique      20110
top       Preface
freq           18
Name: citation_title, dtype: object
count                                                 19521
unique                                                19517
top       © CERN for the benefit of the CMS collaboratio...
freq                                                      3
Name: abstracts, dtype: object
count            20175
unique            1107
top       Elsevier Ltd
freq              1638
Name: publishername, dtype: object
count                  20186
unique                  5500
top       Scientific Report

# Data Cleaning

In [88]:
#Drop useless columns to remove unsuitable data
df_clean = df.drop(columns=["file_id","citation_title","abstracts","document_classification_codes"])
df_clean

,year,publishername,sourcetitle,publication_date,refcount,citedbycount,authors_deg_name_json,categories,creator,keywords
0,2018,Springer International Publishing,"Radiology in Global Health: Strategies, Implem...",31/12/2018,76,1.0,"[""PhD.. Krit Pongpirul"",""VE. Matthew P. Lungren""]","[{""Medicine (all)"":""MEDI""}]",Krit Pongpirul,[]
1,2018,Institute of Electrical and Electronics Engine...,Progress in Electromagnetics Research Symposium,31/12/2018,4,1.0,"[""Teerapong Pratumsiri"",""Panuwat Janpugdee""]","[{""Electrical and Electronic Engineering"":""ENG...",Teerapong Pratumsiri,[]
2,2018,Elsevier Ltd,Chemical Engineering Science,31/12/2018,42,21.0,"[""Kiattikhoon Phuakpunk"",""Benjapon Chalermsins...","[{""Chemistry (all)"":""CHEM""},{""Chemical Enginee...",Kiattikhoon Phuakpunk,"[""Circulating fluidized bed"",""Computational fl..."
3,2018,Elsevier B.V.,Applied Surface Science,31/12/2018,45,37.0,"[""Jittraporn Saengkaew"",""Duy Le"",""Chanatip Sam...","[{""Chemistry (all)"":""CHEM""},{""Condensed Matter...",Jittraporn Saengkaew,"[""Encapsulation"",""Fluoroalkylsilane"",""Natural ..."
4,2018,Elsevier B.V.,Analytica Chimica Acta,31/12/2018,55,68.0,"[""Prinjaporn Teengam"",""Weena Siangproh"",""Adiso...","[{""Analytical Chemistry"":""CHEM""},{""Biochemistr...",Prinjaporn Teengam,"[""acpcPNA"",""Electrochemical impedance spectros..."
...,...,...,...,...,...,...,...,...,...,...
20181,2023,Elsevier B.V.,Catalysis Today,01/01/2023,63,3.0,"[""Duy Le"",""Nattaporn Chaidherasuwet"",""Atitarn ...","[{""Catalysis"":""CENG""},{""Chemistry (all)"":""CHEM""}]",Duy Le,"[""Long-chain olefins"",""Mesoporous KIT-6"",""Olei..."
20182,2023,Taylor and Francis Ltd.,Critical Reviews in Analytical Chemistry,None,115,11.0,"[""Waleed Alahmad"",""Puttaruksa Varanusupakul"",""...","[{""Analytical Chemistry"":""CHEM""}]",Waleed Alahmad,"[""Biological hazards"",""chemical hazards"",""food..."
20183,2023,Routledge,Compare,None,76,5.0,"[""Tejendra Pherali""]","[{""Education"":""SOCI""}]",Tejendra Pherali,"[""conflict"",""Education"",""peacebuilding"",""socia..."
20184,2023,Taylor and Francis Ltd.,Journal of Applied Aquaculture,None,44,6.0,"[""Ratchaneegorn Mapanao"",""Wirat Jiwyam"",""Nudth...","[{""Ecology"":""ENVI""},{""Aquatic Science"":""AGRI""}]",Ratchaneegorn Mapanao,"[""Anabas testudineus"",""Black soldier fly"",""fis..."


In [89]:
#Check for na values and number of rows and columns remaining
print(df_clean.shape)
df_clean.isna().sum()


(20186, 10)


year                        0
publishername              11
sourcetitle                 0
publication_date         4105
refcount                  411
citedbycount                4
authors_deg_name_json       0
categories                  0
creator                     0
keywords                    0
dtype: int64

In [90]:
#Cleaning for 'publishername'
print(df_clean.publishername.unique().tolist())
#All values looks fine, none count will be dealt with by imputer


['Springer International Publishing', 'Institute of Electrical and Electronics Engineers Inc.', 'Elsevier Ltd', 'Elsevier B.V.', 'Czestochowa University of Technology', 'American Chemical Society', 'Association for Computing Machinery', 'Institute of Physics Publishing', 'Magnolia Press', 'American Physical Society', 'Wiley-VCH Verlag', 'Academy of Taiwan Information Systems Research', 'MDPI AG', 'John Wiley and Sons Inc.', 'Springer Netherlands', 'American Institute of Physics Inc.', 'Association for Computing Machinery, Inc', 'IEEE Computer Society', 'BioMed Central Ltd.', 'Taylor and Francis', 'Emerald Group Holdings Ltd.', 'American Society for Microbiology', 'Chulalongkorn University', 'Emerald Group Publishing Ltd.', 'Chulalongkorn University Printing House', 'Springer Nature', 'Thammasat University', 'Oxford University Press', 'Current Medicine Group LLC 1', 'Gemological Institute of America (GIA)', 'Rynnye Lyan Resources', 'Association of Agricultural Technology in Southeast As

In [91]:
#Cleaning for 'sourcetitle'
print(df_clean.sourcetitle.unique().tolist())
#Value looks fine

['Radiology in Global Health: Strategies, Implementation, and Applications', 'Progress in Electromagnetics Research Symposium', 'Chemical Engineering Science', 'Applied Surface Science', 'Analytica Chimica Acta', 'Polish Journal of Management Studies', 'Gene', 'Essentials of Interventional Cancer Pain Management', 'Proceedings of the 5th Asian Conference on Defence Technology, ACDT 2018', 'Langmuir', 'Surface and Coatings Technology', 'ISCIT 2018 - 18th International Symposium on Communication and Information Technology', 'ACM International Conference Proceeding Series', 'Astrophysical Journal', 'IOP Conference Series: Earth and Environmental Science', 'Phytotaxa', 'Physical Review Letters', 'ChemSusChem', 'Journal of Cleaner Production', 'Contemporary Management Research', 'Journal of Physics: Conference Series', 'Molecules', 'Physical Review D', 'Journal of Polymer Science, Part B: Polymer Physics', 'Engineering Structures', 'Journal of Molecular Liquids', 'BioControl', 'Journal of A

In [92]:
#Cleaning with publicationdate

In [93]:
#Cleaning for refcount. Refcount is dtype object so must be converted to int
df_clean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20186 entries, 0 to 20185
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   year                   20186 non-null  int64  
 1   publishername          20175 non-null  object 
 2   sourcetitle            20186 non-null  object 
 3   publication_date       16081 non-null  object 
 4   refcount               19775 non-null  object 
 5   citedbycount           20182 non-null  float64
 6   authors_deg_name_json  20186 non-null  object 
 7   categories             20186 non-null  object 
 8   creator                20186 non-null  object 
 9   keywords               20186 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 1.5+ MB
